<a href="https://colab.research.google.com/github/VallabhK/SentimentExtraction/blob/master/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sentiment Extraction**
This is the program which parses the tweets from Twitter using python and other open source libraries

In [2]:
#Importing all the required libraries
import tweepy #Access twitter API 
from textblob import TextBlob #Process text data
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [11]:
url = 'https://raw.githubusercontent.com/VallabhK/SentimentExtraction/master/config.csv?token=AIB6MQIKVUDYIUELZW2T3SDA64LPO'
config = pd.read_csv(url)

In [17]:
twitterApiKey = config['twitterApiKey'][0]
twitterApiKeySecret = config['twitterApiSecret'][0]
twitterApiAccessToken = config['twitterApiAccessToken'][0]
twitterApiAccessTokenSecret = config['twitterApiAccessTokenSecret'][0]

In [19]:
#Create authentication object
authenticate = tweepy.OAuthHandler(twitterApiKey, twitterApiKeySecret)

#Set access token and secret
authenticate.set_access_token(twitterApiAccessToken, twitterApiAccessTokenSecret)

#Create the API object while passing in auth information
api = tweepy.API(authenticate, wait_on_rate_limit=True)